In [1]:
import time, datetime
from datetime import datetime, timedelta
import csv
import calendar
import pandas as pd
import numpy.random as random
import sys

In [2]:
data = pd.read_csv("data.csv")

#multiply to get time in seconds
data['time'] = data['time']*8

In [3]:
#for i in data['time']:
    #print Index(data['time'],i.get_loc()
timeSeries = list(data['time'])

for i in range(0,len(timeSeries)):
        if i > 0:
            if timeSeries[i] < timeSeries[i-1]:
                #recursively edit overflow
                timeSeries[i] = timeSeries[i-1] + 8
                print "edited due to overflow or reset at row " + str(i)

#write data back into dataframe
data['time'] = timeSeries

edited due to overflow or reset at row 43
edited due to overflow or reset at row 44
edited due to overflow or reset at row 45
edited due to overflow or reset at row 46
edited due to overflow or reset at row 47
edited due to overflow or reset at row 48
edited due to overflow or reset at row 49
edited due to overflow or reset at row 50
edited due to overflow or reset at row 51
edited due to overflow or reset at row 52
edited due to overflow or reset at row 53
edited due to overflow or reset at row 54
edited due to overflow or reset at row 55
edited due to overflow or reset at row 56
edited due to overflow or reset at row 57
edited due to overflow or reset at row 58
edited due to overflow or reset at row 59
edited due to overflow or reset at row 60
edited due to overflow or reset at row 61
edited due to overflow or reset at row 62
edited due to overflow or reset at row 63
edited due to overflow or reset at row 64
edited due to overflow or reset at row 65
edited due to overflow or reset at

In [4]:
def singleSync(series):
    #obtain how many seconds elapsed 
    lastReading = data[series][len(data)-1]

    calculatedStartDate = datetime.now()- timedelta(seconds=int(lastReading))
    #load times as timedelta
    timedeltas = pd.to_timedelta(data[series],unit='s')
    columnName = str('realtime - ' + series)
    #adjust in time since epoch the starting time of the loggings
    data[columnName] = calculatedStartDate + timedeltas

    #time string formatting
    data[columnName] = pd.to_datetime(pd.DatetimeIndex(data[columnName]).strftime('%X %D'))

In [5]:
singleSync('time')
#generate some noise
def noise():
    return random.randint(-2,2) * random.rand()

timeSeries = list(data['time'])
#generate a fake user-inputted start date with some error built-in (for testing time correction purposes only

for i in range (0,len(timeSeries)):
        #if i == 0:
         #       timeSeries[i] = timeSeries[i] + noise()
        if i > 0:
                timeSeries[i] = timeSeries[i-1] + 8 + noise()

data['noisytime'] = timeSeries

singleSync('noisytime')


In [6]:
def getUserStartDate():
    while True:
        #userIn = raw_input("Type Date: mm/dd/yy: ")
        userDateInput = raw_input("enter the day the sensor was turned on and placed in the nest as mm/dd/yyyy ")
        try:
            date = datetime.strptime(userDateInput, "%m/%d/%Y")
            #date = datetime.date(year=date.year, month = date.month, day=date.day)
        except ValueError as e:
            print "Invalid Format: {0}".format(e)
        else:
            return date

def getUserStartTime():
    while True:
        #do the biologists use 24hr time?
        userTimeInput = raw_input("enter the time the sensor was turned on and placed in the nest as 24-Hour time in the format hr:min (e.x. 16:32): ")
        try:
            time = datetime.strptime(userTimeInput + ":00", "%X")
            #time = timedelta(hours=time.hour, minutes=time.minute, seconds=0)
        except ValueError as e:
            print "Invalid Format: {0}".format(e)
        else:
            return time

In [7]:
#from http://code.activestate.com/recipes/577058/
def query_yes_no(question, default="yes"):
    """Ask a yes/no question via raw_input() and return their answer.

    "question" is a string that is presented to the user.
    "default" is the presumed answer if the user just hits <Enter>.
        It must be "yes" (the default), "no" or None (meaning
        an answer is required of the user).

    The "answer" return value is True for "yes" or False for "no".
    """
    valid = {"yes": True, "y": True, "ye": True,
             "no": False, "n": False}
    if default is None:
        prompt = " [yes/no] "
    elif default == "yes":
        prompt = " [yes/no] "
    elif default == "no":
        prompt = " [yes/no] "
    else:
        raise ValueError("invalid default answer: '%s'" % default)

    while True:
        sys.stdout.write(question + prompt)
        choice = raw_input().lower()
        if default is not None and choice == '':
            return valid[default]
        elif choice in valid:
            return valid[choice]
        else:
            sys.stdout.write("Please respond with 'yes' or 'no' "
                             "(or 'y' or 'n').\n")

In [ ]:
def valiDate():
    date = getUserStartDate()
    time = getUserStartTime()
    userStartDateTime = datetime(year=date.year, month=date.month, day=date.day, hour=time.hour, minute= time.minute, second=0)
    print "your date is: " + str(userStartDateTime)
    userVerify = query_yes_no("is this correct? ")
    if userVerify == False:
        valiDate()
    if userVerify == True:
        print "synchronizing time now..."
        return userStartDateTime


def DoubleSync(series):
    columnName = str('doubleSyncRealTime - ' + series)
    #if the user start date doesn't match the singleSync start date, do I adjust by taking the
    #currentTime - userStartTime / number of readings (not preserving 8-sec logging) or by doing
    #userStartTime + 8n?

    #% will adjust syncing approach based on the percentage off the data is.
    tolerance = .1
    userStartDateTime = valiDate()
    timeElapsed = data[series][len(data[series])-1] - data[series][0]
    if userStartDateTime - data[series][0] !=0:
        percentOff = abs((userStartDateTime - data[series][0])/(data[series][len(data[series])-1] - data[series][0]))
        print percentOff
        if percentOff > tolerance:
            #userStartTime + 8n
            data['doublesync'] = pd.date_range(freq=pd.DateOffset(seconds=8),start=userStartDateTime,periods=len(data[series]))
            for i in range(0, len(data[series])-1):
                print userStartDateTime + timedelta(seconds=8*i)
                #data['doublesync'].append(userStartDateTime + timedelta(0,0, 8*i ))
            data.head()
        elif percentOff < tolerance:
            #print "wow, you're actually pretty accurate"
            #interval = 8 seconds +/- some number of microseconds
            interval = (data[series][len(data[series])-1] - userStartDateTime)/(len(data[series]) - 1)
            print str(interval) + "seconds"
            data['doublesync'] = pd.date_range(start=userStartDateTime, periods= len(data[series]), freq=pd.DateOffset(seconds=interval.seconds, microseconds=interval.microseconds))


#data['doublesync'].append(datetime(2017,10,27,8,59,0))
DoubleSync('realtime - noisytime')

In [37]:
data

time  temp  humidity         realtime - time   noisytime  \
0      8  29.6        50 2017-02-25 12:47:19.152    8.000000   
1     16  29.6        34 2017-02-25 12:47:27.152   16.609524   
2     24  28.6        62 2017-02-25 12:47:35.152   24.481961   
3     32  30.6        76 2017-02-25 12:47:43.152   32.317719   
4     40  31.6        80 2017-02-25 12:47:51.152   39.745483   
5     48  30.6        38 2017-02-25 12:47:59.152   48.026363   
6     56  31.6        41 2017-02-25 12:48:07.152   55.236816   
7     64  28.6        78 2017-02-25 12:48:15.152   61.413762   
8     72  31.6        35 2017-02-25 12:48:23.152   68.358074   
9     80  31.6        86 2017-02-25 12:48:31.152   75.659252   
10    88  28.6        76 2017-02-25 12:48:39.152   83.103777   
11    96  28.6        44 2017-02-25 12:48:47.152   90.803103   
12   104  27.6        55 2017-02-25 12:48:55.152   97.442947   
13   112  30.6        75 2017-02-25 12:49:03.152  105.079689   
14   120  30.6        82 2017-02-25 12:49:11.152  113.079689   
15   128  30.6        85 2017-02-25 12:49:19.152  120.656433   
16   136  31.6        38 2017-02-25 12:49:27.152  128.224702   
17   144  30.6        34 2017-02-25 12:49:35.152  135.252248   
18   152  30.6        36 2017-02-25 12:49:43.152  143.252248   
19   160  27.6        92 2017-02-25 12:49:51.152  151.252248   
20   168  27.6        73 2017-02-25 12:49:59.152  158.806818   
21   176  29.6        53 2017-02-25 12:50:07.152  167.505671   
22   184  30.6        78 2017-02-25 12:50:15.152  176.340749   
23   192  28.6        36 2017-02-25 12:50:23.152  184.047493   
24   200  31.6        75 2017-02-25 12:50:31.152  192.596835   
25   208  28.6        64 2017-02-25 12:50:39.152  200.596835   
26   216  30.6        76 2017-02-25 12:50:47.152  208.084019   
27   224  31.6        82 2017-02-25 12:50:55.152  216.084019   
28   232  30.6        54 2017-02-25 12:51:03.152  223.196147   
29   240  30.6        86 2017-02-25 12:51:11.152  231.924487   
..   ...   ...       ...                     ...         ...   
46   376  27.6        70 2017-02-25 12:53:27.152  363.299821   
47   384  30.6        70 2017-02-25 12:53:35.152  372.031177   
48   392  27.6        43 2017-02-25 12:53:43.152  380.485877   
49   400  31.6        78 2017-02-25 12:53:51.152  387.674291   
50   408  31.6        67 2017-02-25 12:53:59.152  396.249883   
51   416  28.6        85 2017-02-25 12:54:07.152  404.849817   
52   424  29.6        40 2017-02-25 12:54:15.152  411.991514   
53   432  28.6        95 2017-02-25 12:54:23.152  419.416115   
54   440  30.6        56 2017-02-25 12:54:31.152  427.197459   
55   448  27.6        43 2017-02-25 12:54:39.152  435.582985   
56   456  27.6        73 2017-02-25 12:54:47.152  442.649564   
57   464  27.6        91 2017-02-25 12:54:55.152  450.649564   
58   472  27.6        70 2017-02-25 12:55:03.152  459.332087   
59   480  28.6        40 2017-02-25 12:55:11.152  467.471190   
60   488  27.6        84 2017-02-25 12:55:19.152  474.952900   
61   496  27.6        86 2017-02-25 12:55:27.152  482.398677   
62   504  28.6        78 2017-02-25 12:55:35.152  488.844079   
63   512  29.6        45 2017-02-25 12:55:43.152  496.844079   
64   520  27.6        75 2017-02-25 12:55:51.152  504.844079   
65   528  29.6        79 2017-02-25 12:55:59.152  512.844079   
66   536  31.6        79 2017-02-25 12:56:07.152  521.650741   
67   544  27.6        72 2017-02-25 12:56:15.152  530.258771   
68   552  27.6        52 2017-02-25 12:56:23.152  538.258771   
69   560  28.6        62 2017-02-25 12:56:31.152  545.419748   
70   568  29.6        87 2017-02-25 12:56:39.152  553.419748   
71   576  27.6        47 2017-02-25 12:56:47.152  561.419748   
72   584  30.6        37 2017-02-25 12:56:55.152  570.102901   
73   592  29.6        92 2017-02-25 12:57:03.152  577.762666   
74   600  27.6        36 2017-02-25 12:57:11.152  585.762666   
75   608  29.6        94 2017-02-25 12:57:19.152  593.790732   

            realtime - noisytime   

In [10]:
testing = pd.DatetimeIndex(start = datetime(2017,2,24,10,40, freq="S")
testing

SyntaxError: invalid syntax (<ipython-input-10-37bf1c50c8e9>, line 2)

In [ ]:
?pd.date_range()